# Network models comparison

Here I will compare a real world network, with two models generated using the same number of nodes and average degree. The testable aim is to get a sense of how different are the models from a real world network.

## Notation used to characterize different network properties

The following measures are useful to characterize local and global properties of a network:


  $z_i$ := degree of node $i$, where $i=1, 2, \ldots, N$ nodes.
  
  $C_i$ := cluster coefficient of node $i$.
  
  $C_{net}$ := the clustering coefficient of the network ($\neq  \langle C \rangle$).
  
   $\langle d_i \rangle$ := $\frac{1}{N-1} \sum_{j \ne i}^N d_{i,j}$, the average geodesic distance from node $i$ to all other nodes $j$.
 
 $d_{diam}$ := $\max_{i,j} d_{i,j}$, the diameter of the network ($\neq \langle \langle d \rangle \rangle := \frac{1}{N} \sum_{j=1}^N \langle d_j \rangle$).
    
$\langle \ldots \rangle$ is the average over all $N$ nodes of the network (i.e., $\langle f \rangle = \frac{1}{N} \sum_{i=1}^N f_i$).

### Let's import the libraries

In [1]:
%matplotlib inline
import sys
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx


<font color="blue"> 
<br> I'm interested in comparing how an empirical network, let's say $G$, compares to random graph models fitted to it. The netwrok I picked represents a school of dolphins, and more information can be found here: 
    
Ssocial network of dolphins. According to the description, the nodes represent to each dolphin and links represent frequent association between them. 

I picked the dataset from: <link>http://networkrepository.com/soc-dolphins.php</link>. 

Some features:

    Nodes = 62
    Edges = 159
    Maximum degree = 12
    Minimum degree = 1
    # of connected components = 1

</font>

In [9]:
# Reading the file where the network is stored

G_original=nx.read_adjlist("./soc-dolphins.txt",nodetype=int) # Political Blogs, undirected

### Actual network

For my actual netwrok, I will compute the following measures: $N,\langle z \rangle, \sigma_z:=\sqrt{\langle z^2 \rangle - \langle z \rangle^2}, C_{net}, \langle \langle d \rangle \rangle, d_{diam}$.

In [14]:
#Some functions that are going to be useful in the development of the analysis

def average_degree(deg):
    
    """
    Input
    deg: list of degrees get from a given network. Tipically, it is obtained using the following operation
    
    list(G.degree()). The function degree in networkx generates a set of degrees from the network.
    
    Output
    average_deg: Average degree of the given network
    std_deg: Standard deviation of the degree distribution
    
    """
    degrees = [elem[1] for elem in deg]
    average_deg = np.average(degrees)    
    std_deg = np.std(degrees)
    
    return average_deg, std_deg


def erdos_connected_component(G):
    
    """
    Input
    G: a network, let's say G. 
    
    
    Output
    dp_avg: Average shortest path of the network
    diam_p: Diameter of the network (shortest maximum distance)
    Boolean: True or False depending on the connectedness of the network. Let's say: 
    If it has one (1) connected component, returns True.
    If it is a disconnected network, namely, it has more than one (1) connected component, returns False.
    
    """
    
    if nx.is_connected(G):
        dp_avg = nx.average_shortest_path_length(G)
        diam_p = nx.diameter(G)
        return dp_avg, diam_p, True
    
    elif not nx.is_connected(G):
        largest_cc = max(nx.connected_components(G))
        Gcc = G.subgraph(largest_cc)  
        dp_avg = nx.average_shortest_path_length(Gcc)
        diam_p = nx.diameter(Gcc)
        return dp_avg, diam_p, False



In [11]:
N = nx.number_of_nodes(G_original)
G_degrees = list(G_original.degree())

G_central = average_degree(G_degrees)

print("The number of nodes is {}".format(N))
print("The average degree is {:.02f}.".format(G_central[0]))
print("The standard deviation of the average degree is {:.03f}.".format(G_central[1]))

C_net = nx.transitivity(G_original)
print("The <<C_net>> coefficient is {:.03f}.".format(C_net))

d_avg = nx.average_shortest_path_length(G_original)
print("The average shortest path is {:.02f}.".format(d_avg))

diam = nx.diameter(G_original)
print("The diameter is {}.".format(diam))

The number of nodes is 62
The average degree is 5.13.
The standard deviation of the average degree is 2.932.
The <<C_net>> coefficient is 0.309.
The average shortest path is 3.36.
The diameter is 8.


### Poisson Random Graph

<p> Let $G_P$ be the Poisson Random Graph fitted to $G$ (i.e., both $G$ and $G_P$ have the same $N$ and $\langle z \rangle$).
    For $G_P$ (Poisson RG), I will compute:  $\langle z \rangle, \sigma_z:=\sqrt{\langle z^2 \rangle - \langle z \rangle^2}, C_{net}, \langle \langle d \rangle \rangle, d_{diam}$.

In [16]:
q = G_central[0]/(N-1)
n = 80 #number of realizations of the Poisson Random Graph 

avg_deg_Gp = []
std_dev_Gp = []
Cp_net_iter = []

# These are the lists for connected components
dp_con_avg_list = []
diam_con_p_list = []
count_conn = 0

# These are the lists for the largest component when the resultant graph isn't connected 
dp_disc_avg_list = []
diam_disc_p_list = []
count_disc = 0

for i in range (n):
    Gp = nx.erdos_renyi_graph(N,q)
    Np = nx.number_of_nodes(Gp)
    Gp_degrees = list(Gp.degree())

    Gp_central = average_degree(Gp_degrees)
    avg_deg_Gp.append(Gp_central[0])
    std_dev_Gp.append(Gp_central[1])

    Cp_net = nx.transitivity(Gp)
    Cp_net_iter.append(Cp_net)

    Gconnec = erdos_connected_component(Gp)

    if Gconnec[2]:
        dp_con_avg_list.append(Gconnec[0])
        diam_con_p_list.append(Gconnec[1])
        count_conn += 1

    elif not Gconnec[2]:
        dp_disc_avg_list.append(Gconnec[0])
        diam_disc_p_list.append(Gconnec[1])
        count_disc += 1
    
print("The average degree is {:.03f} +/- {:.03f} standard deviations."\
      .format(np.average(avg_deg_Gp),np.std(avg_deg_Gp)))

print("The average standard deviation is {:.03f} +/- {:.03f} standard deviations."\
      .format(np.average(std_dev_Gp),np.std(std_dev_Gp)))

print("The average <<C>> is {:.03f} +/- {:.03f} standard deviations."\
      .format(np.average(Cp_net_iter),np.std(Cp_net_iter)))

print("")
print("There were {} number of connected graps in the {} realizations of the Poisson Random Graph"\
      .format(count_conn,n))
print("")
print("The average shortest path over realizations is {:.03f} +/- {:.03f} standard deviations."\
      .format(np.average(dp_con_avg_list),np.std(dp_con_avg_list)))
print("The average diameter is {:.03f} +/- {:.03f} standard deviations."\
      .format(np.average(diam_con_p_list),np.std(diam_con_p_list)))

print("")
print("There were {} non-connected graps in the {} realizations".format(count_disc,n))
print("")
print("The average shortest path over realizations is {:.03f} +/- {:.03f} standard deviations."\
      .format(np.average(dp_disc_avg_list),np.std(dp_disc_avg_list)))

print("The average diameter is {:.03f} +/- {:.03f} standard deviations "\
      .format(np.average(diam_disc_p_list),np.std(diam_disc_p_list)))


The average degree is 5.191 +/- 0.345 standard deviations.
The average standard deviation is 2.135 +/- 0.208 standard deviations.
The average <<C>> is 0.084 +/- 0.016 standard deviations.

There were 60 number of connected graps in the 80 realizations of the Poisson Random Graph

The average shortest path over realizations is 2.638 +/- 0.089 standard deviations.
The average diameter is 5.117 +/- 0.551 standard deviations.

There were 20 non-connected graps in the 80 realizations

The average shortest path over realizations is 2.513 +/- 0.582 standard deviations.
The average diameter is 4.800 +/- 1.249 standard deviations 


### Configuration model

<p> Let $G_C$ be the Configuration Model fitted to $G$ (i.e., both $G$ and $G_P$ have the same $N$ and and the same degree sequence).
    For $G_C$, I'll compute:  $\langle z \rangle, \sigma_z:=\sqrt{\langle z^2 \rangle - \langle z \rangle^2}, C_{net}, \langle \langle d \rangle \rangle, d_{diam}$.
    
If the reader is interested on the Configuration Model, in the following link they can find a good introduction: https://en.wikipedia.org/wiki/Configuration_model

In [17]:
print("For Gc we have the following measures: ")

G_degrees = list(G_original.degree())
deg_G = [elem[1] for elem in G_degrees] #creating a list with every single degree
deg_G.sort()


avg_deg_Gc = []
std_dev_Gc = []
Ccc_net_iter = []

# 1 single connected component graph realization from the generated graph
dcc_avg_list = []
diam_cc_list = []
Cc_realizations = 0

# Largest connected component list from a non-connected realization of the generated graph
dcc_notcon_avg_list = []
diam_notcon_list = []
notCc_realizations = 0

for i in range(n):
    
    Gcc=nx.configuration_model(deg_G)
    Gcc_degrees = list(Gcc.degree())
    avg_deg = average_degree(Gcc_degrees)
    
    avg_deg_Gc.append(avg_deg[0])
    std_dev_Gc.append(avg_deg[1])
    
    G=nx.Graph(Gcc)
    Ccc_net = nx.transitivity(G)
    Ccc_net_iter.append(Ccc_net)
    
    G.remove_edges_from(nx.selfloop_edges(G))
    Gcc_connec = erdos_connected_component(G)
    
    if Gcc_connec[2]:
        dcc_avg_list.append(Gcc_connec[0])
        diam_cc_list.append(Gcc_connec[1])
        Cc_realizations += 1
    
    else:
        dcc_notcon_avg_list.append(Gcc_connec[0])
        diam_notcon_list.append(Gcc_connec[1])
        notCc_realizations += 1
        
print("The average degree is {:.02f} +/- {:.02f} standard deviations."\
      .format(np.average(avg_deg_Gc),np.std(avg_deg_Gc)))
print("The average standard deviation is {:.03f} and its std is {:.03f}."\
      .format(np.average(std_dev_Gc),np.std(std_dev_Gc)))


print("The average <<C>> is {:.03f} and its standard deviation is {:.03f}"\
      .format(np.average(Ccc_net_iter),np.std(Ccc_net_iter)))

print("")
print("For 1 single connected component from different realizations, we have")
print("The average shortest path over n = {} realizations is {:.03f} +/- {:.03f} standard deviations."\
      .format(Cc_realizations,np.average(dcc_avg_list),np.std(dcc_avg_list)))
print("The average diameter is {:.03f} +/- {:.03f} standard deviations."\
      .format(np.average(diam_cc_list),np.std(diam_cc_list)))

print("")
print("For those realizations that are not a single component, we have:")
print("The average shortest path over n = {} realizations is {:.03f} +/- {:.03f} standard deviations."\
      .format(notCc_realizations,np.average(dcc_notcon_avg_list),np.std(dcc_notcon_avg_list)))
print("The average diameter is {:.03f} +/- {:.03f} standard deviations."\
      .format(np.average(diam_notcon_list),np.std(diam_notcon_list)))


For Gc we have the following measures: 
The average degree is 5.13 +/- 0.00 standard deviations.
The average standard deviation is 2.932 and its std is 0.000.
The average <<C>> is 0.101 and its standard deviation is 0.017

For 1 single connected component from different realizations, we have
The average shortest path over n = 66 realizations is 2.801 +/- 0.050 standard deviations.
The average diameter is 5.879 +/- 0.508 standard deviations.

For those realizations that are not a single component, we have:
The average shortest path over n = 14 realizations is 2.273 +/- 0.758 standard deviations.
The average diameter is 4.500 +/- 2.096 standard deviations.


## Explantion of the results

Comparing the values obtained for $G$, $G_P$, and $G_C$, discussing if the values are larger, smaller, or (approximately) equal. In each case, I give an interpretation and explanation of the numerical results (e.g., is it  a trivial consequence of the construction of the model?).
    
(a) <b>$\langle z \rangle$</b>

Actual network =  5.13<br>
Poisson RG (average over 80 realizations) = $5.08 \pm 0.353$<br>
Configuration model (average over 80 realizations) =$ 5.13 \pm 0.00$ standard deviations<br>

<b>Explanation</b><br>
As expected, the averages are not too different between each other, since it was the fixed measure to form the PRG and the Configuration model. One interesting thing to note is that the standard deviation on every single realization of the Configuration model is 0, which means that the algorithm is going to assign the same degree sequence no matter if for each new iteration we are creating a new instance of the graph.  

<br>

(b) $\sigma_z:=\sqrt{\langle z^2 \rangle - \langle z \rangle^2}$

Actual network =  2.932.<br>
Poisson RG (average over 80 realizations) = $2.133 \pm 0.213$ <br>
Configuration model = $2.932 \pm 0.00$ <br>

<b>Explanation</b><br>
As expected, the average degrees of the original network and the Configuration models are the same, because the model was generated from the degree sequence of the original graph. So in concordance, their standard deviations are the same as well, even over the whole set of simulations of the Configuration model. 

In the case of the PRG, we got a value of $2.13 \pm 0.198$, which is close to the value of $\sqrt[2] {\langle z \rangle}  = \sqrt[2] {5.18} = 2.276$. It is necessary to take into account that we have several realizations of the PRG and therefore, we are introducing variability. Nevertheless, with this calculation we are inside the range defined.


(c) $C_{net}$

 
Actual network = 0.309<br>
Poisson RG (average over 80 realizations) = 0.084 $\pm$ 0.019<br>
Configuration model = 0.098 $\pm$ 0.016<br>

<b>Explanation</b><br>
As expected, the $C_{net}$ coefficient of the original graph is much larger than the average $C_{net}$ coefficient for the 80 runs of the PRG and the realizations of the Configuration Model. It is expected because in this random models, the probability of form triangles is small. In both cases, the value of the average over realizations is close to $\frac{\langle z \rangle}{N-1}$, which agrees with the value we analyzed in the lecture for the clustering coefficient. I think that this N = 62 is large enough to observe this trend. 



(d) $\langle \langle d \rangle \rangle$

Actual network = 3.36<br>

Generated Graph with just one single component.<br>
Poisson RG (average over 80 realizations) = 2.670 $\pm$ 0.106<br>
Configuration model = 2.815 $\pm$ 0.046<br>

Generated Graph with more than one component. The treatment here was to create a new subgraph for the largest component.<br>
Poisson RG (average over 80 realizations) = 2.646 $\pm$ 0.098<br>
Configuration model = 2.268 $\pm$ 0.787 <br>

<b>Explanation</b><br>

I think it is expected to get a shortest path lower than the actual one for PRG and the Generated Model. Since there's sub-polynomial growth for the diameter in both models, is somehow expected that the other important measure of distance is smaller than the observed for the actual case. 



(e) $d_{diam}$
    
ctual network = 8<br>
Generated Graphs with just one single component.<br>
Poisson RG (average over 80 realizations) = 5.260 $\pm$ 0.559<br>
Configuration model = 6.031 $\pm$ 0.632<br>

Generated Graphs with more than one component. The treatment here was to create a new subgraph for the largest component.<br>
Poisson RG (average over 80 realizations) = $5.333 \pm 0.471$<br>
Configuration model = 4.933 $\pm$ 2.048 <br>


<b>Explanation</b><br>
As expected, the diameters of the Configuration Model and the PRG are lower than the actual diameter. And this is repeated in both cases -when the resultant model is either a connected or a non-connected graph. I think N is not big enough, but for bigger N's, we should expect that $ d_{diam} \:\alpha \: ln(N) $, and $ln(N)\:=\:ln(62)\:=\:4.1271$. The good news is that for the average diameter $\pm$ standard deviation, we got a smaller diameter than the one we got from the original graph.<br> 
The closest case is when we get a smaller graph through the subgraph function to transform the multi-graph (the one obtained in the Configuration Model), to a simple one. However, I think it is not enough evidence to conclude and we should try to get a graph with a larger N. 
<br>

<b>Last reflection</b><br>

We could see that the configuration model behaves identically to the actual network for the degree distribution, the average and its standard deviation. Obviously, this is because it uses the same degree sequence of G (original network) and hence, reproduces its variability. 
However, for the measures $C_{net}$, $\langle d \rangle$ and $d_{diam}$, it behaves like a PRG -or similarly. As long as it is a sort of generalization of the PRG, we should expect the it behaves in a similar manner for those measures, which are the ones that best characterize the dynamics in a RG.  
    
        
        